<a href="https://colab.research.google.com/github/whdid502/stt_model_project/blob/feature_extraction/pcm_to_mfcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tensorflow 1.13.2 버전 필요

In [34]:
!pip install tensorflow==1.13.2

In [35]:
!pip3 install SpecAugment

In [36]:
import wave
import librosa
import IPython.display as ipd
# from matplotlib import pyplot as plt
import numpy as np
import argparse
import librosa.display
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from specAugment import spec_augment_tensorflow
from scipy.fftpack import dct
import os

In [37]:
import matplotlib
matplotlib.use('TKAgg')

pcm to mfcc

In [38]:
def pcm_to_mfcc(audio_path, noise_injection):

  signal = np.memmap(audio_path, dtype='h', mode='r').astype('float32') # load audio
  data = signal / 32767   # normalize audio

  if noise_injection:           # noise injection
    wn = np.random.randn(len(data))
    data_wn = data + 0.005*wn
  else:
    data_wn = data
  
  sr = 16000
  mel_spectrogram = librosa.feature.melspectrogram(y=data_wn, sr=sr, n_mels=256, hop_length=128, fmax=8000) # data to melspectrogram

  warped_masked_spectrogram = spec_augment_tensorflow.spec_augment(mel_spectrogram=mel_spectrogram) # melspectrogram spec augmentation

  num_ceps = 12
  mfcc = dct(warped_masked_spectrogram, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # mel spectrogram to mfcc

  return mfcc

In [39]:
output = pcm_to_mfcc('/content/drive/My Drive/googledrive/feature_test/KsponSpeech_000001.pcm', True)

In [40]:
output.shape

(256, 12)

pcmfile 싹 가져오기

In [41]:
total_list = []
def pcm_list(root_dir):
    files = os.listdir(root_dir)
    for file in files:
        path = os.path.join(root_dir, file)

        if os.path.isdir(path):
            pcm_list(path)
        total_list.append(path)

    file_list_pcm = [file for file in total_list if file.endswith(".pcm")]
    return file_list_pcm

In [42]:
pcm_li = pcm_list('/content/drive/My Drive/googledrive/feature_test')
print(pcm_li)
print(len(pcm_li))

['/content/drive/My Drive/googledrive/feature_test/KsponSpeech_000001.pcm', '/content/drive/My Drive/googledrive/feature_test/KsponSpeech_000002.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373004.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373007.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373006.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373003.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373002.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373001.pcm', '/content/drive/My Drive/googledrive/feature_test/test_folder_for_feature/KsponSpeech_373005.pcm']
9


In [57]:
mfcc_list = []
# train_count = 1

def training(pcm):
  mfcc_list.append(pcm_to_mfcc(pcm, False))
  # print(train_count + '번째 훈련 시작')
  print('인코더')
  print('디코더')
  print('훈련')
  print('체크포인트')
  print('현재 mfcc변환 개수 : ' + str(len(mfcc_list)))
  # print(train_count) 

def training_process(pcm_li, batch_size):
  convert_count = 0
  if len(pcm_li) < batch_size:
    for pcm in pcm_li:
      training(pcm)
      
  else:
    for pcm in pcm_li:
      training(pcm)
      convert_count += 1
      if convert_count == batch_size:
        break
  if batch_size <= len(pcm_li):
    next_list = pcm_li[batch_size:]
    print('남은 pcm 개수 : ' + str(len(next_list)))
    training_process(next_list, batch_size)

In [58]:
training_process(pcm_li,3)
# print(len(mfcc_list))
# print(len(pcm_li))
len(mfcc_list)

인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 1
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 2
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 3
남은 pcm 개수 : 6
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 4
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 5
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 6
남은 pcm 개수 : 3
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 7
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 8
인코더
디코더
훈련
체크포인트
현재 mfcc변환 개수 : 9
남은 pcm 개수 : 0


9

In [63]:
mfcc_list[2]

array([[-1.4893614e-01, -9.0331399e-01, -5.0934637e-01, ...,
         4.5932502e-01,  5.0939888e-02,  1.9189888e-01],
       [-1.9615728e-02, -4.1044268e-01, -1.2822321e-01, ...,
         2.4337146e-01,  9.3780465e-02,  5.7286665e-02],
       [ 1.1023341e-02, -3.5794504e-02,  9.7195044e-02, ...,
         5.0411329e-02,  1.2286048e-01,  9.0374619e-02],
       ...,
       [-3.2053138e-03, -5.9474446e-03,  6.8540974e-03, ...,
         2.1928856e-03,  1.0191158e-03, -2.6170702e-03],
       [-2.0517674e-03, -3.5864846e-03,  4.3518217e-03, ...,
         1.2334632e-03,  3.5185338e-04, -1.3425916e-03],
       [-1.5453136e-03, -2.7536158e-03,  3.3146660e-03, ...,
         1.1062571e-03, -1.3210310e-04, -8.7890454e-04]], dtype=float32)